In [3]:

import sys
sys.path.append(r"D:\8th Semester\Machine learning\ML Labs")

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [5]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [6]:
def CNN():
    input_data = Input(shape=(time_steps, num_features))
    x1 = Conv1D(16, 2, activation="relu")(input_data)
    x2 = Conv1D(16, 2, activation="relu")(x1)
    flatten = Flatten()(x2)
    output_data = Dense(1)(flatten)
    model = Model(input_data, output_data)
    return model

In [7]:
model1 = CNN()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 23, 16)              │             688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 22, 16)              │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 352)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             353 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,569 (6.13 KB)

 Trainable params: 1,569 (6.13 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [34]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab8\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'D:\8th Semester\Machine learning\ML Labs\lab8'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [36]:
import os
from keras.callbacks import ModelCheckpoint

# Define necessary paths and variables
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

FIG_PATH = os.path.sep.join([output_dir, "training_plot.png"])
JSON_PATH = os.path.sep.join([output_dir, "training_history.json"])
checkpoints = os.path.join(r'D:\8th Semester\Machine learning\ML Labs\lab8', 'best_model.h5')

start_epoch = 0  # Change this if you're resuming from a checkpoint

# Construct the callback to save only the *best* model to disk based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                                   monitor="val_loss",
                                   save_best_only=True, 
                                   verbose=1)

# Make sure the TrainingMonitor class is implemented/imported properly
#TrainingMonitor1 = TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# Construct the set of callbacks
callbacks = [EpochCheckpoint1]


In [37]:
import os
from keras.models import load_model
from keras.optimizers import Adam
from keras.losses import MeanAbsoluteError
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError
import keras.backend as K

# Define your model path
model_path = r'D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5'

# Define your CNN architecture (dummy here, replace with your own if needed)
def CNN():
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D(pool_size=(2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1)  # Regression output
    ])
    return model

# If the model doesn't exist
if not os.path.exists(model_path):
    print("[INFO] compiling new model...")
    model = CNN()
    opt = Adam(learning_rate=1e-3)
    model.compile(loss=MeanAbsoluteError(), optimizer=opt,
                  metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()])

# Load model with proper custom_objects mapping
else:
    print(f"[INFO] loading model from {model_path}...")

    model = load_model(
        model_path,
        custom_objects={
            'mae': MeanAbsoluteError(),  # This fixes the deserialization issue
            'MeanAbsoluteError': MeanAbsoluteError,
            'MeanAbsolutePercentageError': MeanAbsolutePercentageError
        }
    )
import tensorflow as tf

print("[INFO] old learning rate: {}".format(model.optimizer.learning_rate.numpy()))
model.optimizer.learning_rate.assign(1e-4)
print("[INFO] new learning rate: {}".format(model.optimizer.learning_rate.numpy()))

[INFO] loading model from D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [38]:
import os
path_dataset =r'D:\8th Semester\Machine learning\ML Labs\DATASET'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [39]:
time_steps=24
num_features=21

In [40]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.46247243881225586 sec


In [41]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [42]:
from keras.optimizers import Adam
from keras.losses import MeanAbsoluteError
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError

model.compile(
    optimizer=Adam(learning_rate=1e-4),  # fresh optimizer
    loss=MeanAbsoluteError(),
    metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()]
)


In [43]:
# Unfreeze model if needed
model.trainable = True
for layer in model.layers:
    layer.trainable = True

# Recompile with correct loss
from keras.losses import MeanAbsoluteError
from keras.metrics import MeanAbsoluteError as MAE

model.compile(
    optimizer=Adam(1e-4),
    loss=MeanAbsoluteError(),
    metrics=[MAE()]
)

# Debug shapes
print("train_X shape:", train_X.shape)
print("train_y shape:", train_y.shape)
print("Model input shape:", model.input_shape)
print("Model output shape:", model.output_shape)
print("Trainable variables:", len(model.trainable_variables))

# Fit the model
History = model.fit(
    train_X, train_y,
    batch_size=32,
    epochs=epochs,
    validation_data=(validation_X, validation_y),
    callbacks=callbacks,
    verbose=1
)


train_X shape: (84882, 24, 21)
train_y shape: (84882, 1)
Model input shape: (None, 24, 21)
Model output shape: (None, 1)
Trainable variables: 6


C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/5
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0077 - mean_absolute_error: 0.0077  
Epoch 1: val_loss improved from inf to 0.00804, saving model to D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 128s 48ms/step - loss: 0.0077 - mean_absolute_error: 0.0077 - val_loss: 0.0080 - val_mean_absolute_error: 0.0080
Epoch 2/5
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0075 - mean_absolute_error: 0.0075  
Epoch 2: val_loss improved from 0.00804 to 0.00787, saving model to D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 127s 48ms/step - loss: 0.0075 - mean_absolute_error: 0.0075 - val_loss: 0.0079 - val_mean_absolute_error: 0.0079
Epoch 3/5
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0075 - mean_absolute_error: 0.0075  
Epoch 3: val_loss improved from 0.00787 to 0.00768, saving model to D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 126s 47ms/step - loss: 0.0075 - mean_absolute_error: 0.0075 - val_loss: 0.0077 - val_mean_absolute_error: 0.0077
Epoch 4/5
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0074 - mean_absolute_error: 0.0074  
Epoch 4: val_loss did not improve from 0.00768
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 128s 48ms/step - loss: 0.0074 - mean_absolute_error: 0.0074 - val_loss: 0.0079 - val_mean_absolute_error: 0.0079
Epoch 5/5
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0074 - mean_absolute_error: 0.0074  
Epoch 5: val_loss improved from 0.00768 to 0.00761, saving model to D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 132s 50ms/step - loss: 0.0074 - mean_absolute_error: 0.0074 - val_loss: 0.0076 - val_mean_absolute_error: 0.0076


In [44]:

model = load_model(r'D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
Mean Absolute Error (MAE): 123.43
Median Absolute Error (MedAE): 101.21
Mean Squared Error (MSE): 26568.44
Root Mean Squared Error (RMSE): 163.0
Mean Absolute Percentage Error (MAPE): 0.85 %
Median Absolute Percentage Error (MDAPE): 0.69 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [48]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab8\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5'
start_epoch= 58

In [52]:
import os
import tensorflow as tf
from keras.models import load_model
from keras.optimizers import Adam
from keras.losses import MeanAbsoluteError
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError

# Define your model path
model_path = r"D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5"

# Check if model file exists
if not os.path.exists(model_path):
    print(f"[ERROR] Model file not found at: {model_path}")
else:
    print(f"[INFO] Loading model from: {model_path}")

    # Load the model without compiling (to avoid deserialization issues)
    model = load_model(model_path, compile=False)

    # Recompile with current Keras 3-compatible objects
    model.compile(
        loss=MeanAbsoluteError(),
        optimizer=Adam(learning_rate=1e-4),
        metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()]
    )

    # Show updated learning rate
    print("[INFO] Updated learning rate:", model.optimizer.learning_rate.numpy())

    # Optional: show summary
    model.summary()


[INFO] Loading model from: D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5
[INFO] Updated learning rate: 1e-04


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_10 (Conv1D)                   │ (None, 23, 16)              │             688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 22, 16)              │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 352)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             353 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,569 (6.13 KB)

 Trainable params: 1,569 (6.13 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2651/2653 [============================>.] - ETA: 0s - loss: 0.0064 - mae: 0.0064 - mape: 711.7792
Epoch 1: val_loss improved from inf to 0.00659, saving model to C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab8\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 36s 13ms/step - loss: 0.0064 - mae: 0.0064 - mape: 711.3610 - val_loss: 0.0066 - val_mae: 0.0066 - val_mape: 4.3460
Epoch 2/10
2644/2653 [============================>.] - ETA: 0s - loss: 0.0063 - mae: 0.0063 - mape: 578.1265
Epoch 2: val_loss improved from 0.00659 to 0.00642, saving model to C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab8\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 10s 4ms/step - loss: 0.0063 - mae: 0.0063 - mape: 576.2668 - val_loss: 0.0064 - val_mae: 0.0064 - val_mape: 4.2309
Epoch 3/10
2646/2653 [============================>.] - ETA: 0s - loss: 0.0063 - mae: 0.0063 - mape: 551.6437
Epoch 3: val_loss did not improve from 0.0064

In [54]:

model = load_model(r'D:\8th Semester\Machine learning\ML Labs\lab8\best_model.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

  6/379 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
Mean Absolute Error (MAE): 123.43
Median Absolute Error (MedAE): 101.21
Mean Squared Error (MSE): 26568.44
Root Mean Squared Error (RMSE): 163.0
Mean Absolute Percentage Error (MAPE): 0.85 %
Median Absolute Percentage Error (MDAPE): 0.69 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
